# Backtest and Optimiser !!!
1. This script can backtest and optimise any single position strategy.
2. The script uses a csv from Binance spot BTC/USDT, please ensure you have a suitable CSV before attempting to read in a dataframe.
3. When testing/optimising your strategy, it is a good idea to mask the data to a smaller sample set to confirm accuracy before completing a full backtest. As a rough example to optimise 5 timeframes, with two moving averages, and four fixed TP and SL options i.e 740 iterations of Backtest() could take over 8 hours!

In [33]:
#Libraries
import pandas as pd
import pandas_ta as ta
import numpy as np
import itertools
#import plotly.graph_objects as go

In [34]:
#READING AND CLEANING + SET INDEX
df = pd.read_csv("../Binance_BTCUSDT_1min.csv")
df = df.iloc[:,:6]
df.columns=['Date','Open', 'High', 'Low', 'Close', 'Volume']
df.reset_index(drop=True, inplace=True)
df.Date = pd.to_datetime(df.Date)
df = df.set_index("Date")
df

,Open,High,Low,Close,Volume
Date,,,,,
2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183
2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000
2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074
2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008
2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796
...,...,...,...,...,...
2023-08-30 22:03:00,27271.14,27312.57,27271.13,27307.48,59.000970
2023-08-30 22:04:00,27307.47,27326.87,27307.47,27314.21,57.145400
2023-08-30 22:05:00,27314.21,27314.22,27311.32,27311.33,9.068380


In [35]:
#MASKING TO DECREASE SAMPLE SIZE
start_date = "2022-01-01 00:00:00"
end_date = "2024-01-01 00:00:00"
mask = (df.index > start_date) & (df.index <= end_date)
df=df.loc[mask]
df

,Open,High,Low,Close,Volume
Date,,,,,
2022-01-01 00:01:00,46250.00,46344.23,46234.39,46312.76,42.38106
2022-01-01 00:02:00,46312.76,46381.69,46292.75,46368.73,51.29955
2022-01-01 00:03:00,46368.73,46391.49,46314.26,46331.08,30.45894
2022-01-01 00:04:00,46331.07,46336.10,46300.00,46321.34,20.96029
2022-01-01 00:05:00,46321.34,46443.56,46280.00,46436.03,35.86682
...,...,...,...,...,...
2023-08-30 22:03:00,27271.14,27312.57,27271.13,27307.48,59.00097
2023-08-30 22:04:00,27307.47,27326.87,27307.47,27314.21,57.14540
2023-08-30 22:05:00,27314.21,27314.22,27311.32,27311.33,9.06838


In [36]:
#Function for resampling the dataframe
def resample_df(df, freq):
    resampled_open = df.Open.resample(freq).first()
    resampled_high = df.High.resample(freq).max()
    resampled_low = df.Low.resample(freq).min()
    resampled_close = df.Close.resample(freq).last()
    resampled_volume = df.Volume.resample(freq).sum()
    new_df = pd.concat([resampled_open, resampled_high, resampled_low, resampled_close, resampled_volume], axis=1)
    new_df.dropna(inplace=True)
    return new_df

In [37]:
def calc_price(new_df):
    new_df["price"] = new_df.Open.shift(-1)

In [7]:
def calc_vwap(new_df, anchor_period):
    new_df['VWAP'] = ta.vwap(new_df.High, new_df.Low, new_df.Close, new_df.Volume, anchor=anchor_period)

In [38]:
def calc_isHigh(new_df, n):
    new_df["highest_close"] = new_df.High.rolling(n).max()
    new_df["isHigh"] = new_df.High > new_df.highest_close.shift(1) 

In [39]:
def calc_buy_signal(new_df):
    new_df["buy_signal"] = np.where(((new_df.isHigh > 0) & (new_df.isHigh.shift(1) < 1)), True, False)

In [6]:
#ADDING COLUMNS AND SIGNALS

#def calc_vol(new_df, x):
    #new_df["volma"] = new_df.Volume.rolling(x).mean()

#def calc_bb(new_df, n, std):
    #bb_df = ta.bbands(new_df.Close, n, std)
    #bb_df.columns=['bblower', 'bbmid', 'bbupper', 'bandwidth', 'bbperc',]
    #new_df = pd.merge(new_df, bb_df, on='Date', how='inner')
    #return new_df

#def calc_macd(new_df, fast, slow, signal):
    #macd_df = ta.macd(new_df.Close, fast, slow, signal)
    #macd_df.columns=['macd', 'macd_histogram', 'macd_signal']
    #new_df = pd.merge(new_df, macd_df, on='Date', how='inner')
    #return new_df

#def calc_ret(new_df):
    #new_df["ret"] = (new_df.Close.shift(-1)/new_df.Close)-1

#def calc_vwap(new_df, anchor_period):
    #new_df['VWAP'] = ta.vwap(new_df.High, new_df.Low, new_df.Close, new_df.Volume, anchor=anchor_period)
    
#def calc_ma(new_df, n):
    #new_df['SMA'] = new_df.Close.rolling(n).mean()

#def calc_rsi(new_df, rsi_length, rsiSMA_period):
    #new_df["rsi"] = ta.rsi(new_df.Close, length=rsi_length).round(2)
    #new_df["rsiSMA"] = new_df.rsi.rolling(rsiSMA_period).mean().round(2)
    #new_df['rsi_std'] = new_df.rsi.rolling(20).std(ddof=0)*2
    #new_df["rsibbUpper"] = (new_df.rsiSMA + new_df.rsi_std).round(2)
    #new_df["rsibbLower"] = (new_df.rsiSMA - new_df.rsi_std).round(2)

In [53]:
def backtest(df, freq, n, tp, sl):

    new_df = resample_df(df, freq)
    calc_isHigh(new_df, n)
    calc_price(new_df)
    calc_buy_signal(new_df)
    new_df.dropna(inplace=True)
    
    #Error handle for no buy signals
    if len(new_df[new_df.buy_signal > 0]) < 1:
        empty_result = pd.DataFrame({
            "entry_time": [0],
            "entry_price": [0],
            "tp_target": [0],
            "sl_target": [0],
            "exit_time": [0],
            "exit_price": [0],
            "pnl": [0],
            "equity": [0]
        })
        amount = 0
        winrate = 0
        pnl = 0
        equity = 0
        return amount, winrate, pnl, equity
        return empty_result
    
    #Initialise Varibles
    in_position = False
    trades = []
    current_trade = {}
    account_size = 1000
            
    for i in range(len(new_df)-1):
    #Check exit conditions
        if in_position:
            if new_df.iloc[i].Low < current_trade["sl_price"]:
                current_trade["exit_price"] = current_trade["sl_price"]
                pnl = (current_trade["exit_price"] - current_trade["entry_price"]) / current_trade["entry_price"]
                trades.append({
                    "entry_time":current_trade["entry_time"],
                    "entry_price":current_trade["entry_price"],
                    "tp_target":current_trade["tp_price"],
                    "sl_target":current_trade["sl_price"],
                    "exit_time":new_df.iloc[i].name,
                    "exit_price":current_trade["sl_price"],
                    "pnl": pnl,
                    "equity": account_size + (account_size * pnl)
                })
                account_size = account_size + (account_size * pnl)
                current_trade = {}
                in_position = False

            elif new_df.iloc[i].High > current_trade["tp_price"]:
                current_trade["exit_price"] = current_trade["tp_price"]
                pnl = (current_trade["exit_price"] - current_trade["entry_price"]) / current_trade["entry_price"]
                trades.append({
                    "entry_time":current_trade["entry_time"],
                    "entry_price":current_trade["entry_price"],
                    "tp_target":current_trade["tp_price"],
                    "sl_target":current_trade["sl_price"],
                    "exit_time":new_df.iloc[i].name,
                    "exit_price":current_trade["tp_price"],
                    "pnl":pnl,
                    "equity": account_size + (account_size * pnl)
                })
                account_size = account_size + (account_size * pnl)
                current_trade = {}
                in_position = False

        #Check entry conditions
        if not in_position:
            if new_df.iloc[i].buy_signal == True:
                current_trade["entry_price"] = new_df.iloc[i].price
                current_trade["entry_time"] = new_df.iloc[i+1].name
                current_trade["tp_price"] = new_df.iloc[i].price*tp
                current_trade["sl_price"] = new_df.iloc[i].price*sl
                #current_trade["base_value"] = trade_amount/new_df.iloc[i].price
                #current_trade["quote_value"] = trade_amount
                in_position = True
                
    data = pd.DataFrame(trades)
    amount = len(data)
    winrate = len(data.loc[data.pnl.values>0])/len(data)*100
    pnl = sum(pd.Series(data.pnl))
    equity = data.equity[len(data)-1]
    return amount, winrate, pnl, equity
    return pd.DataFrame(trades)

In [54]:
#def backtest(df, freq, n, tp, sl):

In [55]:
backtest(df, "4H", 10, 1.04, 0.96)

(121, 45.45454545454545, -0.43999999999999956, 584.4334194015487)

NameError: name 'DD' is not defined

# Optimiser

In [59]:
# Define the parameter combinations
freq_values = ["5T", "1H", "4H", "1D"]
n_values = [5, 7, 8, 10, 20, 50]
tp_values = [1.02, 1.03, 1.04, 1.09]
sl_values = [0.98, 0.97, 0.96, 0.91]

In [60]:
#def backtest(df, freq, n, tp, sl):

In [61]:
#Create an empty dataframe
results_df = pd.DataFrame(columns=["freq", "n","tp", "sl", "amount", "winrate", "pnl", "equity"])

In [ ]:
for freq, n, tp, sl in itertools.product(freq_values, n_values, tp_values, sl_values):
    amount, winrate, pnl, equity = backtest(df, freq, n, tp, sl)
    result = pd.DataFrame([[freq, n, tp, sl, amount, winrate, pnl, equity]], columns=["freq", "n", "tp", "sl", "amount", "winrate", "pnl", "equity"])
    results_df = pd.concat([results_df, result], ignore_index=True)

# Print the results DataFrame
print(results_df)

In [ ]:
results_df

In [33]:
#results_df[results_df.pnl.max()]
results_df["rr"] = (results_df.tp - 1) / (1 - results_df.sl)

In [35]:
filtered_results = results_df[(results_df["winrate"] > 50) & (results_df["amount"] > 300) & (results_df["rr"]>1) ]
filtered_results

,freq,anchor_period,x,vol_ratio,tp,sl,amount,winrate,pnl,rr
318,1H,D,50,3.0,1.03,0.98,353,50.991501,1.94,1.500000
426,1H,W,50,3.0,1.03,0.98,321,52.959502,2.08,1.500000
574,4H,W,20,1.5,1.04,0.97,407,50.368550,2.14,1.333333


In [38]:
results_df[results_df.pnl>3]

,freq,anchor_period,x,vol_ratio,tp,sl,amount,winrate,pnl,rr
149,5T,W,20,2.0,1.03,0.96,1328,60.617470,3.23,0.75
150,5T,W,20,2.0,1.04,0.98,1608,37.002488,3.54,2.00
162,5T,W,30,1.5,1.02,0.98,3419,52.442235,3.34,1.00
164,5T,W,30,1.5,1.02,0.96,2184,69.001832,3.06,0.50
167,5T,W,30,1.5,1.03,0.96,1523,60.078792,3.13,0.75
168,5T,W,30,1.5,1.04,0.98,1922,36.056191,3.14,2.00
183,5T,W,30,3.0,1.03,0.98,1420,44.295775,3.05,1.50
189,5T,W,50,1.5,1.02,0.98,3446,52.205456,3.04,1.00
191,5T,W,50,1.5,1.02,0.96,2172,69.290976,3.42,0.50
194,5T,W,50,1.5,1.03,0.96,1548,59.948320,3.04,0.75


In [ ]:
results_df.to_csv('HighClose-TrendFollow-Backtest.csv', index=True)

In [66]:
csv_df = pd.read_csv('BTCUSDT_VWAPVOL-Backtest.csv')
csv_df.reset_index(drop=True, inplace=True)
csv_df[(csv_df.freq=="1H") & (csv_df.x==20) & (csv_df.vol_ratio==3)]

,Unnamed: 0,freq,anchor_period,x,vol_ratio,tp,sl,amount,winrate,pnl
261,261,1H,D,20,3.0,1.02,0.98,345,53.623188,0.50
262,262,1H,D,20,3.0,1.02,0.97,341,60.410557,0.07
263,263,1H,D,20,3.0,1.02,0.96,332,67.469880,0.16
264,264,1H,D,20,3.0,1.03,0.98,319,45.454545,0.87
265,265,1H,D,20,3.0,1.03,0.97,310,52.258065,0.42
266,266,1H,D,20,3.0,1.03,0.96,292,59.931507,0.57
267,267,1H,D,20,3.0,1.04,0.98,296,38.513514,0.92
268,268,1H,D,20,3.0,1.04,0.97,283,44.169611,0.26
269,269,1H,D,20,3.0,1.04,0.96,266,52.255639,0.48
369,369,1H,W,20,3.0,1.02,0.98,304,55.263158,0.64


# Visualisation

In [ ]:
#CANDLE STICK AND MERGED PLOTS
fig = go.Figure(data=[go.Candlestick(
    x = new_df.index,
    open = new_df.Open,
    high = new_df.High,
    low = new_df.Low,
    close = new_df.Close,
    increasing_line_color = "rgba(0, 0, 0, 0.5)",
    decreasing_line_color = "rgba(0, 0, 0, 0.5)",
    name="Candlesticks"
    ),
    go.Scatter(x=new_df.index, y=new_df.VWAP, line=dict(color='red', width=1)),
    go.Scatter(x=new_df.index, y=new_df.SMA, line=dict(color='blue', width=1))
])

#ENTRY PLOTS
fig.add_trace(go.Scatter(
    x=trades.entry_time,
    y=trades.entry_price,
    mode = "markers",
    customdata=trades,
    marker_symbol="triangle-up",
    marker_color="rgba(0, 255, 0, 0.9)",
    marker_line_color="rgba(0, 0, 0, 0.5)",
    marker_size =13,
    marker_line_width=1,
    name="Entries",
    hovertemplate="Entry Time: %{customdata[1]}<br>"
        "Entry Price: %{y:.2f}<br>"
        "Size: %{customdata[2]:.6f}"
))

#EXIT PLOTS
fig.add_trace(go.Scatter(
    x=trades.exit_time,
    y=trades.exit_price,
    mode = "markers",
    customdata=trades,
    marker_symbol="triangle-down",
    marker_color="rgba(255, 0, 0, 0.9)",
    marker_line_color="rgba(0, 0, 0, 0.5)",
    marker_size =13,
    marker_line_width=1,
    name="Exits",
    hovertemplate="Exit Time: %{customdata[4]}<br>"
        "Exit Price: %{y:.2f}<br>"
        #"Size: %{customdata[2]:.6f}"
))

#fig.update_layout (xaxis_rangeslider_visable = False)
fig